**Examples of punpy usage**

Imagine you are trying to calibrate some L0 data to L1 and you have:

- A measurement function that uses L0 data, gains, and a dark signal in 5 wavelength bands

- Random uncertainties and systematic uncertainties on the L0 data;

- Random and systematic uncertainties on the gains;

- Random uncertainties on the dark signals

We first import our punpy package and numpy, and define an example measurement function and example data:

In [ ]:
!pip install punpy

In [ ]:
import punpy
import numpy as np

# your measurement function
def calibrate(L0,gains,dark):
   return (L0-dark)*gains

# your data
wavs = np.array([350,450,550,650,750])
L0 = np.array([0.43,0.8,0.7,0.65,0.9])
dark = np.array([0.05,0.03,0.04,0.05,0.06])
gains = np.array([23,26,28,29,31])

# your uncertainties
L0_ur = L0*0.05  # 5% random uncertainty
L0_us = np.ones(5)*0.03  # systematic uncertainty of 0.03
                         # (common between bands)
gains_ur = np.array([0.5,0.7,0.6,0.4,0.1])  # random uncertainty
gains_us = np.array([0.1,0.2,0.1,0.4,0.3])  # systematic uncertainty
# (different for each band but fully correlated)
dark_ur = np.array([0.01,0.002,0.006,0.002,0.015])  # random uncertainty


After defining the data, the resulting uncertainty budget can then be calculated with punpy using the MC methods as:

In [ ]:
prop=punpy.MCPropagation(10000)
L1=calibrate(L0,gains,dark)
L1_ur=prop.propagate_random(calibrate,[L0,gains,dark],
      [L0_ur,gains_ur,dark_ur])
L1_us=prop.propagate_systematic(calibrate,[L0,gains,dark],
      [L0_us,gains_us,np.zeros(5)])
L1_ut=(L1_ur**2+L1_us**2)**0.5
L1_cov=punpy.convert_corr_to_cov(np.eye(len(L1_ur)),L1_ur)+\
       punpy.convert_corr_to_cov(np.ones((len(L1_us),len(L1_us))),L1_ur)

plt.plot(wavs,L1,"o")
plt.errorbar(wavs,L1,yerr=L1_ur,label="random uncertainty")
plt.errorbar(wavs,L1,yerr=L1_us,label="systematic uncertainty")
plt.errorbar(wavs,L1,yerr=L1_ut,label="total uncertainty")
print(L1)
print(L1_ur)
print(L1_us)
print(L1_ut)
print(L1_cov)

We now have for each band the random uncertainties in L1, systematic uncertainties in L1, total uncertainty in L1 and the covariance matrix between bands. Here we have manually specified a diagonal correlation matrix (no correlation, np.eye) for the random component and a correlation matrix of ones (fully correlated, np.ones). It would also have been possible to use the keyword return_corr to get the measured correlation matrix. In the next example we use the LPU methods set the return_corr keyword:


In [ ]:
prop=punpy.MCPropagation(10000)
L1=calibrate(L0,gains,dark)
L1_ur,L1_corr_r=prop.propagate_random(calibrate,[L0,gains,dark],
                [L0_ur,gains_ur,dark_ur],return_corr=True)
L1_us,L1_corr_s=prop.propagate_systematic(calibrate,[L0,gains,dark],
                [L0_us,gains_us,np.zeros(5)],return_corr=True)
L1_ut=(L1_ur**2+L1_us**2)**0.5
L1_cov=punpy.convert_corr_to_cov(L1_corr_r,L1_ur)+\
       punpy.convert_corr_to_cov(L1_corr_s,L1_ur)

print(L1)
print(L1_ur)
print(L1_us)
print(L1_ut)
print(L1_cov)

This will give nearly the same results other than a small error due to MC noise.

Next we give an example where we try out a measurement function with multiple outputs. In order to process a measurement function with multiple outputs, it is necessary to set the keyword output_vars to the number of outputs:

In [ ]:
# your measurement function
def calibrate_2output(L0,gains,dark):
   return (L0-dark)*gains,(L0*gains-dark)

prop=punpy.MCPropagation(10000)
L1=calibrate_2output(L0,gains,dark)
L1_ur,L1_corr_r,L1_corr_r_between=prop.propagate_random(
                                  calibrate_2output,[L0,gains,dark],
                                  [L0_ur,gains_ur,dark_ur],
                                  return_corr=True,output_vars=2)
L1_us,L1_corr_s,L1_corr_s_between=prop.propagate_systematic(
                                  calibrate_2output,[L0,gains,dark],
                                  [L0_us,gains_us,np.zeros(5)],
                                  return_corr=True,output_vars=2)

print(L1)
print(L1_ur)
print(L1_us)

Due to the multiple vars, L1_ur now has the shape (2,5) so L1_ur[0] now has the same uncertainties as the previous example, L1_corr_r[0] is the same as L1_corr_r before. Analogously, L1_ur[1] and L1_corr_r[0] give the random uncertainty and correlation matrix for the second output of the measurand. There is now also a L1_corr_r_between which gives the correlation matrix between the two output variables of the measurment function (averaged over all wavelengths).

In addition to propagating random (uncorrelated) and systematic (fully correlated) uncertainties it is also possible to propagate uncertainties associated with structured errors. If we know the covariance matrix for each of the input quantities, it is straigtforward to propagate these. In the below example we assume the L0 data and dark data to be uncorrelated (their covariance matrix is a, diagonal matrix) and gains to be a custom covariance:

In [ ]:
# your uncertainties
L0_ur = L0*0.05  # 5% random uncertainty
dark_ur = np.array([0.01,0.002,0.006,0.002,0.015])  # random uncertainty

L0_cov=punpy.convert_corr_to_cov(np.eye(len(L0_ur)),L0_ur)
dark_cov=punpy.convert_corr_to_cov(np.eye(len(dark_ur)),dark_ur )
gains_cov= np.array([[0.45,0.35,0.30,0.20,0.05],
                    [0.35,0.57,0.32,0.30,0.07],
                    [0.30,0.32,0.56,0.24,0.06],
                    [0.20,0.30,0.24,0.44,0.04],
                    [0.05,0.07,0.06,0.04,0.21]])


In [ ]:
prop=punpy.MCPropagation(10000)
L1=calibrate(L0,gains,dark)
L1_ut,L1_corr=prop.propagate_cov(calibrate,[L0,gains,dark],
                                 [L0_cov,gains_cov,dark_cov])
L1_cov=punpy.convert_corr_to_cov(L1_corr,L1_ut)

print(L1)
print(L1_ut)
print(L1_cov)